In [1]:
import psycopg2
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
import glob #gather all excel file in folder
from sqlalchemy import create_engine

from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from googleapiclient.discovery import build
import json
import sys

#GOOGLE SHEETS
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
credentials = service_account.Credentials.from_service_account_file('D:\Mia town\IT\Data\study-de-001-10f369c99172.json', scopes = SCOPES)
service = build('sheets', 'v4', credentials=credentials)

#DATABASE
host = 'localhost'
dbname = 'postgres'
user = 'postgres'
password = input('hay nhap pass')
port = '5432'



### POSTGRES DATABASE 

In [3]:
#CREATE TABLE
conn = psycopg2.connect(f'host= {host} \
                        dbname= {dbname} \
                        user= {user} \
                        password= {password} \
                        ')

#set commit automaticaly
conn.set_session(autocommit=True)
cur = conn.cursor()

try:
    cur.execute("""
                CREATE TABLE IF NOT EXISTS product(
                ngay_vao DATE,
                khu_vuc VARCHAR,
                loai_khu_vuc VARCHAR,
                ban VARCHAR,
                ma_hoa_don VARCHAR,
                so_hoa_don VARCHAR,
                ten_hang VARCHAR,
                so_luong FLOAT,
                don_gia INT,
                thanh_tien INT,
                loai_hang VARCHAR);

                CREATE TABLE IF NOT EXISTS invoice(
                khu_vuc VARCHAR,
                loai_khu_vuc VARCHAR,
                ma_hoa_don VARCHAR,
                so_hoa_don VARCHAR,
                ngay_vao DATE,
                gio_vao TIME,
                ngay_ra DATE,
                gio_ra TIME,
                ban VARCHAR,
                tong_hoa_don INT,
                chiet_khau_thanh_toan INT,
                tien_promo INT,
                ten VARCHAR,
                sdt VARCHAR,
                voucher_name VARCHAR);

                CREATE TABLE IF NOT EXISTS expenses(
                khu_vuc VARCHAR,
                loai_chi_phi VARCHAR,
                noi_dung VARCHAR,
                so_tien int,
                ngay_dat_hang DATE);


                """)
except psycopg2.Error as e:
    print('error in creating table')




### REVENUE DATA

In [4]:
def convert_to_datetime(value): #CONVERT TỪ STR THÀNH DATETIME
    if isinstance(value,str):
        return datetime.strptime(value, '%d/%m/%Y')
    elif isinstance(value, datetime):
        return value
    else:
        return None
    
def swap_day_month(value): #ĐỔI VỊ TRÍ NGÀY THÁNG DO BỊ LỆCH FORMAT
    if isinstance(value,datetime):
        return value.replace(day= value.month, month=value.day)
    else: 
        return value

def etl_mia(path):
    ##GET DATA FROM DISK DRIVE
    excel_file = glob.glob(f"{path}/*.xlsx")
    df = []
    for file in excel_file:
        data = pd.read_excel(file, header=1)
        df.append(data)
    df = pd.concat(df, ignore_index=True)   


    #Xử lý dữ liệu Ngày
    df["Ngày vào"] = pd.Series(df["Ngày vào"]).fillna(method = 'ffill')
    df["Ngày vào"] = [swap_day_month(item) for item in df["Ngày vào"]] 
    df["Ngày vào"] = [convert_to_datetime(item) for item in df["Ngày vào"]] 
    df["day_name"] = df["Ngày vào"].dt.day_name()

    df["Ngày ra"] = pd.Series(df["Ngày ra"]).fillna(method = 'ffill') 
    df["Ngày ra"] = [swap_day_month(item) for item in df["Ngày ra"]] 
    df["Ngày ra"] = [convert_to_datetime(item) for item in df["Ngày ra"]] 
    df["day_name"] = df["Ngày ra"].dt.day_name()

    df['Bàn'] = pd.Series(df['Bàn']).fillna(method = 'ffill')
    df['Mã hoá đơn'] = pd.Series(df['Mã hoá đơn']).fillna(method = 'ffill')
    df['Số hoá đơn'] = pd.Series(df['Số hoá đơn']).fillna(method = 'ffill')
    df["loai_khu_vuc"] = df["Bàn"].apply(lambda x: 'CAFE KIDS' if "CAFE KIDS" in x else
                                    'CAFE KIDS' if "Kids Café" in x else   
                                    'BIDA LỖ' if 'BIDA LỖ' in x else
                                    'BIDA BĂNG' if 'BIDA BĂNG' in x else
                                    'BIDA LIBRE' if 'BIDA LIBRE' in x else
                                    'PS5' if 'KHU PS5' in x else
                                    'PS5_ROOM' if 'PS5 ROOM' in x else 
                                    'PS5' if 'PS5' in x else
                                    'PS5' if 'NET' in x else
                                    
                                    'QUẦY VÉ' if "QUẦY VÉ" in x else
                                    'SNACK' if "Bán mang về" in x else None)

    df["khu_vuc"] = df["Bàn"].apply(lambda x: 'CAFE KIDS' if "CAFE KIDS" in x else
                                    'CAFE KIDS' if "Kids Café" in x else   
                                    'BIDA' if 'BIDA LỖ' in x else
                                    'BIDA' if 'BIDA BĂNG' in x else
                                    'BIDA' if 'BIDA LIBRE' in x else
                                    'PS5' if 'KHU PS5' in x else
                                    'PS5' if 'PS5' in x else
                                    'PS5' if 'NET' in x else
                                    'PS5' if 'PS5 ROOM' in x else  
                                    'CAFE KIDS' if "QUẦY VÉ" in x else
                                    'CAFE KIDS' if "Bán mang về" in x else None)

    df['Tên'] = df['Tên'].fillna(value='không có thông tin')
    df['SĐT'] = df['SĐT'].fillna(value='không có thông tin')
    df['Tổng hóa đơn'] = df['Tổng hóa đơn'].fillna(value= 0)
    df['Chiết khấu thanh toán'] = df['Chiết khấu thanh toán'].fillna(value= 0)
    df['Thành tiền\n'] = df['Thành tiền\n'].fillna(value= 0)
    df['Tên Voucher'] = df['Tên Voucher'].fillna(value= 'không sử dụng Voucher')
    # df["Thành tiền\n"] = df["Thành tiền\n"].replace(r'MOMO.*','0', regex=True)
    # df["Thành tiền\n"] = df["Thành tiền\n"].replace(r'TRANSFER.*','0', regex=True)
    # df["Thành tiền\n"] = df["Thành tiền\n"].replace(r'DEPOSIT.*','0', regex=True)
    # df["Thành tiền\n"] = df["Thành tiền\n"].astype(float)

       
    return df


def main_task(path):
    print('Extracting and Transform Data')
    print('--------------------------------')
    data = etl_mia(path)
    invoice, product = creata_data_for_database(data)
    print('Loading Data to Database')
    print('--------------------------------')
    load_data_to_database(invoice, product)
    check_mapping_data(product)
    print('Completed')
# 


### EXPENSES DATA

In [5]:
def get_data_from_ggsheet(sheet_name):
    sheet = service.spreadsheets()
    #lấy data mẫu về để xác định số cột
    result = sheet.values().get(
                spreadsheetId='1GqfHl-mJfLGt6lxEJXfRhhbbxVsogzylajwN2YBgptI',
                range=f'{sheet_name}!A1:1'
            ).execute()

    # lấy số cột
    result = result.get('values', [])
    header = result[0]
    last_column = chr(ord("A") + len(header) - 1)
    range = f'{sheet_name}!A:{last_column}'

    result = sheet.values().get(
                spreadsheetId='1GqfHl-mJfLGt6lxEJXfRhhbbxVsogzylajwN2YBgptI',
                range=range
            ).execute()

    final_df =result.get('values', [])
    final_df = pd.DataFrame(final_df[1:], columns=final_df[0])

    df = final_df[['Khu vực đề xuất', 'Loại chi phí', 'Nội dung đề xuất', 'Số tiền', 'Ngày đặt hàng']].copy()
    df.rename(columns={
        'Khu vực đề xuất':'khu_vuc',
        'Loại chi phí': 'loai_chi_phi',
        'Nội dung đề xuất': 'noi_dung',
        'Số tiền': 'so_tien',
        'Ngày đặt hàng': 'ngay_dat_hang'
    }, inplace=True)
    df.loc[:, 'noi_dung'] = df['noi_dung'].replace('\n', ' ', regex=True)
    df.loc[:,'noi_dung'] = df['noi_dung'].str.lower()
    df.loc[:, 'so_tien'] = df['so_tien'].replace(',', '', regex=True)
    df.loc[:, 'so_tien'] = df['so_tien'].astype(int)
    df.loc[:,'ngay_dat_hang'] = df['ngay_dat_hang'].fillna(value='0')

    return df


def sheet_exists(sheet_name): #dùng để avoid lỗi parse data do 1 năm 12 tháng mà hiện tại chưa tới tháng 12
    try:
        sheet = service.spreadsheets()
        result = sheet.values().get(
            spreadsheetId='1GqfHl-mJfLGt6lxEJXfRhhbbxVsogzylajwN2YBgptI',
            range=f'{sheet_name}!A1:1'
        ).execute()
        return 'values' in result
    except Exception as e:
        return False

def combine_all_expense_data():
    sheet_name = [f"thang{i}/2025" for i in range(1,13)]
    df_list = []

    for j in sheet_name:
        if sheet_exists(j):
            df = get_data_from_ggsheet(j)
            df_list.append(df)
    if df_list:
        combine_df = pd.concat(df_list)
        return combine_df
    else:
        return None
    



### import data vào database

In [6]:
data_mapping = pd.read_excel("D:\Mia town\IT\Data\MAPPING_DATA\data_mapping.xlsx", sheet_name="Sheet1")
## Chỉnh lại chỗ này, chia ra theo từng khu vục
def creata_data_for_database(df):
     ## INVOICE TABLE
    invoice = df[[
        'khu_vuc',
        'loai_khu_vuc',
        'Mã hoá đơn',
        'Số hoá đơn',
        'Ngày vào', 
        'Giờ vào', 
        'Ngày ra', 
        'Giờ ra', 
        'Bàn', 
        'Tổng hóa đơn', 
        'Chiết khấu thanh toán',
        'Thành tiền\n',
        'Tên', 
        'SĐT',
        'Tên Voucher'
        
        ]].dropna()
    invoice.rename(columns={
        'Mã hoá đơn':'ma_hoa_don',
        'Số hoá đơn':'so_hoa_don',
        'Ngày vào':'ngay_vao',
        'Giờ vào':'gio_vao',
        'Ngày ra':'ngay_ra',
        'Giờ ra':'gio_ra',
        'Bàn':'ban',
        'Tổng hóa đơn':'tong_hoa_don',
        'Chiết khấu thanh toán':'chiet_khau_thanh_toan',
        'Thành tiền\n':'tien_promo',
        'Tên':'ten',
        'SĐT':'sdt',
        'Tên Voucher':'voucher_name'
    }, inplace = True)
    invoice['ma_hoa_don'] = invoice['ma_hoa_don'].astype(str) + invoice['so_hoa_don'].astype(str)

    ## PRODUCT TABLE
    product = df[[
        'Ngày vào',
        'khu_vuc',
        'loai_khu_vuc',
        'Bàn',
        'Mã hoá đơn',
        'Số hoá đơn',
        'Tên hàng',
        'Số lượng',
        'Đơn giá'
    ]].dropna()

    product['Tên hàng'] = product['Tên hàng'].str.lower()
    product['thanh_tien'] = product['Số lượng'] * product['Đơn giá']

    product.rename(columns={
        'Ngày vào':'ngay_vao',
        'Bàn':'ban',
        'Mã hoá đơn':'ma_hoa_don',
        'Số hoá đơn':'so_hoa_don',
        'Tên hàng':'ten_hang',
        'Số lượng':'so_luong',
        'Đơn giá':'don_gia',
    }, inplace = True)

    product['ma_hoa_don'] = product['ma_hoa_don'].astype(str) + product['so_hoa_don'].astype(str)
    
    #Mapping data
    product = product.merge(data_mapping, how='left', on='ten_hang')
       
    return invoice, product

def load_data_to_database(invoice, product):
    ## IMPORT DATA TO DATABASE
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')
    
    #filter old vs new data
    existing_invoice = pd.read_sql_table('invoice', engine)
    existing_product_invoice = pd.read_sql_table('product', engine)

    #import new invoice to sql
    new_invoice = invoice[~invoice['ma_hoa_don'].isin(existing_invoice['ma_hoa_don'])]
    new_invoice.to_sql('invoice', engine, if_exists='append', index= False)
    #import new product sale to sql
    new_product = product[~product['ma_hoa_don'].isin(existing_product_invoice['ma_hoa_don'])]
    new_product.to_sql('product', engine, if_exists='append', index= False)
    #import expenses to sql
    # expenses = combine_all_expense_data()
    # expenses.to_sql('expenses', engine, if_exists='replace', index= False)



def check_mapping_data(new_product):
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')
    check = pd.read_sql('product', engine)

    if check['loai_hang'].isnull().any() == True:
        print('Có món chưa gán loại món')
        check = check.loc[:,check.columns != 'loai_hang']
        check = check.merge(data_mapping, how= 'left', on= 'ten_hang')
        print('Mapping xong')
        print('ghi lai len database')
        check.to_sql('product', engine, if_exists='replace', index= False)
        print('done')
    else:
        print('khong co data can map')


### RFM model

In [7]:
def recency_score(recency): 
    if 1 <= recency <= 7: 
        return 4 
    elif 8 <= recency <= 21: 
        return 3 
    elif 22 <= recency <= 30: 
        return 2 
    else: 
        return 1
    
def monetary_score(monetary): 
    if 0 <= monetary <= 150000: 
        return 1 
    elif 150001 <= monetary <= 300000: 
        return 2 
    elif 300001 <= monetary <= 500000: 
        return 3 
    else: 
        return 4
    
def frequency_score(frequency): # chỉ áp dụng 3 tháng/lần chạy model
    if 0 <= frequency <= 6: 
        return 1 
    elif 7 <= frequency <= 9: 
        return 2 
    elif 10 <= frequency <= 12: 
        return 3 
    else: 
        return 4


def rfm_score(df):
    current_date = '28/02/2025'                                             # Chú ý ngày xác nhận báo cáo
    current_date = pd.to_datetime(current_date, format="%d/%m/%Y")
    df['Recency'] = current_date - df['Ngày vào']
    df = df[['SĐT','Tên','khu_vuc','Tổng hóa đơn', 'Recency']].dropna()

    # Create R F M table and merging all
    recency = df.groupby('SĐT')['Recency'].min().reset_index()
    monetary = df.groupby('SĐT')['Tổng hóa đơn'].sum().reset_index().sort_values(by='Tổng hóa đơn', ascending=False)
    frequency = df['SĐT'].value_counts().reset_index()
    rfm_model = recency.merge(monetary, how='left', on='SĐT')
    rfm_model = rfm_model.merge(frequency, how='left', on='SĐT')
    rfm_model.rename(columns={'Tổng hóa đơn': 'Monetary', 'count': 'Frequency'},inplace=True)

    #apply function and provide results
    rfm_model['Recency'] = rfm_model['Recency'].dt.days
    rfm_model['Recency'] = rfm_model['Recency'].astype(int)
    rfm_model['R_score'] = rfm_model['Recency'].apply(recency_score)
    rfm_model['M_score'] = rfm_model['Monetary'].apply(monetary_score)
    rfm_model['F_score'] = rfm_model['Frequency'].apply(frequency_score)
    rfm_model['RFM_score'] = rfm_model['R_score'].astype(str) + rfm_model['F_score'].astype(str) + rfm_model['M_score'].astype(str)
    rfm_model = rfm_model[rfm_model['SĐT'] != 'không có thông tin']

    
   

    return  rfm_model

def categorize_rfm(rfm_model):
    # rfm_model = rfm_score(rfm_model)
    rfm_model['RFM_score'] = rfm_model['RFM_score'].astype(int)

    #Categorize all customer by RFM_Score
    rfm_model['Customer_Type'] = rfm_model['RFM_score'].apply(lambda x: 'VIP' if x in [444,443,434,344] else 
                                                                        'LOYALTY' if x in [442, 441, 432, 431, 433, 343, 342, 341] else
                                                                        'POTENTIAL' if x in [424, 423, 324, 323,413, 414, 343, 334] else
                                                                        'PROMISING' if x in [333, 332, 331, 313, 314] else
                                                                        'NEW' if x in [422, 421, 412, 411, 311, 321, 312, 322] else
                                                                        'PRICE_SENSITIVE' if x in [131, 132, 141, 142, 231, 232, 241, 242] else
                                                                        'NEED_ATTENTION' if x in [244, 234, 243, 233, 224, 214, 213, 134, 144, 143, 133] else
                                                                        'ABOUT_TO_SLEEP' if x in [223, 221, 222, 211,212, 124] else
                                                                        'WALK_IN_CUSTOMER'
                                                                        )
    return rfm_model

def visualize_rfm(path):
    df = etl_mia(path)
    df = df.loc[df['khu_vuc'] == 'BIDA'] # SỬ DỤNG CHO BIDA 
    score = rfm_score(df)
    result = categorize_rfm(score)
    desired_order = ['VIP','LOYALTY','POTENTIAL','PROMISING','NEW','PRICE_SENSITIVE', 'NEED_ATTENTION', 'ABOUT_TO_SLEEP', 'WALK_IN_CUSTOMER']
    chart = result['Customer_Type'].value_counts().reset_index()
    chart['Customer_Type'] = pd.Categorical(chart['Customer_Type'], categories=desired_order, ordered=True) 
    chart = chart.sort_values('Customer_Type')

    plt.figure(figsize=(10, 7)) 
    plt.barh(chart['Customer_Type'], chart['count'], color='skyblue') 
    plt.title('Distribution of RFM Scores') 

    plt.xlabel('Count') 
    plt.ylabel('RFM Score') 
    plt.grid(axis='x', linestyle='--', alpha=0.7)

    plt.show()


### thực hành

In [8]:
# path = 'D:/Mia town/IT/Data/MIATOWN_ALL_DATA'
# main_task(path)

In [31]:
df = pd.read_excel("D:\\Mia town\\IT\\Data\\MIATOWN_ALL_DATA\\gamingps_t04_2025.xlsX", header=1, sheet_name="Sheet1")
df_1 = pd.read_excel("D:\\Mia town\\IT\\Data\\MIATOWN_ALL_DATA\\gamingps_t03_2025.xlsX", header=1, sheet_name="Sheet1")

In [32]:
df.head()

,STT,Cửa hàng,Mã ca,Mã hoá đơn,Số hoá đơn,Số hoá đơn điện tử,Ngày vào,Giờ vào,Ngày ra,Giờ ra,...,SĐT,Địa chỉ,Ghi chú,Tổng hóa đơn,Mã giảm giá\n,Unnamed: 33,Phương thức thanh toán,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Tên Voucher,Thành tiền\n,Tên PTTT,Mã đối tác\n,Số hoá đơn đối tác\n,Thành tiền\n
1,1.0,GAMING+PS,#AG9XN,#MKQF5,310682.0,NaN,30/04/2025,23:30:00,30/04/2025,23:57:00,...,NaN,NaN,Khách k cho sdt,40250.0,NaN,NaN,COD (Z7VZM4P82KG75IVIH7BMKQF5),NaN,NaN,40250
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,GAMING+PS,#AG9XN,#79BKK,310631.0,NaN,30/04/2025,21:31:00,30/04/2025,23:56:00,...,8.490878e+10,Tphcm,NaN,225800.0,NaN,NaN,MOMO_QR_AIO (Z7VZM4P82KG75IVI1U079BKK),86037791051,NaN,225800


In [33]:
df_1.head()

,STT,Cửa hàng,Mã ca,Mã hoá đơn,Số hoá đơn,Số hoá đơn điện tử,Ngày vào,Giờ vào,Ngày ra,Giờ ra,...,SĐT,Địa chỉ,Ghi chú,Tổng hóa đơn,Tên Voucher,Thành tiền\n,Tên PTTT,Mã đối tác\n,Số hoá đơn đối tác\n,Thành tiền\n.1
0,1.0,GAMING+PS,#6JH0G,#JEJJY,320281.0,NaN,31/03/2025,23:26:00,31/03/2025,23:27:00,...,8.486557e+10,57 lê thúc hoạch,NaN,22400.0,GIẢM GIÁ NHÂN VIÊN,-5600.0,MOMO_QR_AIO (Z7VZM4P82KG7459E22OJEJJY),8.312630e+10,NaN,22400.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,GAMING+PS,#6JH0G,#DMF19,320258.0,NaN,31/03/2025,21:08:00,31/03/2025,23:21:00,...,8.483404e+10,728 nguyễn trãi,NaN,198200.0,NaN,NaN,MOMO_QR_AIO (Z7VZM4P82KG7EXC6AODMF19),8.312630e+10,NaN,198200.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df.rename(columns={
    'Mã giảm giá\n': 'Tên Voucher',
    'Unnamed: 33': 'Thành tiền\n',
    'Unnamed: 35': 'Mã đối tác\n',
    'Unnamed: 36': 'Số hoá đơn đối tác\n',
    'Unnamed: 37': 'Thành tiền\n.1'
}, inplace=True)
df = df.drop([0])

In [35]:
df.head()

,STT,Cửa hàng,Mã ca,Mã hoá đơn,Số hoá đơn,Số hoá đơn điện tử,Ngày vào,Giờ vào,Ngày ra,Giờ ra,...,SĐT,Địa chỉ,Ghi chú,Tổng hóa đơn,Tên Voucher,Thành tiền\n,Phương thức thanh toán,Mã đối tác\n,Số hoá đơn đối tác\n,Thành tiền\n.1
1,1.0,GAMING+PS,#AG9XN,#MKQF5,310682.0,NaN,30/04/2025,23:30:00,30/04/2025,23:57:00,...,NaN,NaN,Khách k cho sdt,40250.0,NaN,NaN,COD (Z7VZM4P82KG75IVIH7BMKQF5),NaN,NaN,40250
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.0,GAMING+PS,#AG9XN,#79BKK,310631.0,NaN,30/04/2025,21:31:00,30/04/2025,23:56:00,...,8.490878e+10,Tphcm,NaN,225800.0,NaN,NaN,MOMO_QR_AIO (Z7VZM4P82KG75IVI1U079BKK),86037791051,NaN,225800
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
